In [208]:
import pandas as pd
from collections import Counter, defaultdict
import tensorflow as tf
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [212]:
folder_name = "/home/sanala/Juputer try/PET/data"
save_name="/home/sanala/Juputer try/PET/data-fsl"
for i in range(12):
    language = ['amh','eng','fra','hau','ibo','lin','pcm','run','swa','yor','sna','orm'][i]
 
    train = pd.read_csv(f'{folder_name}/{language}/train.tsv',sep='\t')
    dev= pd.read_csv(f'{folder_name}/{language}/dev.tsv',sep='\t')
    test= pd.read_csv(f'{folder_name}/{language}/test.tsv',sep='\t')
    train_nodup=train.drop_duplicates(subset=['headline'], keep='first')
    dev_nodup=dev.drop_duplicates(subset=['headline'], keep='first')
    classes = train['category'].unique()
    samples_per_class=50 #5 10 20 50
    # create an empty list to store the randomly selected samples
    random_samples = []

    # iterate through each class
    for class_name in classes:
        # select all rows with the current class
        class_rows = train_nodup[train_nodup['category'] == class_name]
        # randomly select X rows from the class_rows DataFrame
        random_class_samples = class_rows.sample(samples_per_class)
        # add the randomly selected samples to the list
        random_samples.append(random_class_samples)
    random_samples_df = pd.concat(random_samples).sample(frac=1)
    new_train=random_samples_df.reset_index(drop=True)
    unlabeled_train = train.drop(new_train.index)
    
    random_samples = []
    samples_per_class=5
    for class_name in classes:
        class_rows = dev_nodup[dev_nodup['category'] == class_name]
        random_class_samples = class_rows.sample(samples_per_class)
        random_samples.append(random_class_samples)
    random_samples_df = pd.concat(random_samples).sample(frac=1)
    new_dev=random_samples_df.reset_index(drop=True)
    
    unlabeled_train.insert(0, 'unlabeled', 'unlabeled')
    unlabeled_train.reset_index(drop=True)
    unlabeled_train.drop(columns='category', inplace=True)
    
    new_train.to_csv(f'{save_name}/{language}/50sample/train.tsv', sep="\t", index=False  ,header=False)
    unlabeled_train.to_csv(f'{save_name}/{language}/50sample/unlabeled.tsv', sep="\t", index=False,header=False)
    new_dev.to_csv(f'{save_name}/{language}/50sample/dev.tsv', sep="\t", index=False,header=False)

#new_train



In [214]:
 
for i in range(12):
    language = ['amh','eng','fra','hau','ibo','lin','pcm','run','swa','yor','sna','orm'][i]
 
    train = pd.read_csv(f'{folder_name}/{language}/train.tsv',sep='\t')
    dev= pd.read_csv(f'{folder_name}/{language}/dev.tsv',sep='\t')
    test= pd.read_csv(f'{folder_name}/{language}/dev.tsv',sep='\t')
    print(language)
    print(train['category'].value_counts())

#new_train


amh
health      350
politics    350
sports      329
business    282
Name: category, dtype: int64
eng
sports           700
politics         574
business         559
entertainment    525
health           522
technology       429
Name: category, dtype: int64
fra
health        350
sports        350
politics      350
business      350
technology     76
Name: category, dtype: int64
hau
entertainment    350
politics         350
sports           347
religion         345
health           345
business         279
technology       203
Name: category, dtype: int64
ibo
politics         350
health           296
entertainment    256
business         204
sports           199
religion          51
Name: category, dtype: int64
lin
politics    350
health      135
sports       66
business     57
Name: category, dtype: int64
pcm
sports           344
entertainment    322
politics         216
health           111
business          67
Name: category, dtype: int64
run
politics         350
sports           293
h

In [228]:

train = pd.read_csv('/home/sanala/Juputer try/PET/data/eng/train.tsv', sep="\t")
dev = pd.read_csv('/home/sanala/Juputer try/PET/data/eng/dev.tsv', sep="\t")
test = pd.read_csv('/home/sanala/Juputer try/PET/data/eng/test.tsv', sep="\t")

train.info()
train.head(10)
dev.info()     

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3309 entries, 0 to 3308
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  3309 non-null   object
 1   headline  3309 non-null   object
 2   text      3309 non-null   object
 3   url       3309 non-null   object
dtypes: object(4)
memory usage: 103.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 472 entries, 0 to 471
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  472 non-null    object
 1   headline  472 non-null    object
 2   text      472 non-null    object
 3   url       472 non-null    object
dtypes: object(4)
memory usage: 14.9+ KB


In [229]:
#There are repeated examples in the training dataset
print('Unique comments: ', train.iloc[:,1].nunique()==train.shape[0] )
print('Null values: ', train.isnull().values.any())
     

Unique comments:  False
Null values:  False


In [230]:
train['category'].value_counts()

sports           700
politics         574
business         559
entertainment    525
health           522
technology       429
Name: category, dtype: int64

In [231]:
#Usually the headlines are sort but outliers can appear.
print('average sentence length: ', train.iloc[:,1].str.split().str.len().mean())
print('stdev sentence length: ', train.iloc[:,1].str.split().str.len().std())
print('max sentence length: ', train.iloc[:,1].str.split().str.len().max())

average sentence length:  9.527954064672107
stdev sentence length:  2.053447292397227
max sentence length:  22


In [233]:
train[train.iloc[:,2].duplicated(keep=False).values]

,category,headline,text,url
734,sports,Commonwealth Games: Table Tennis - Men's Doubl...,Results from preliminary rounds can be found o...,/sport/commonwealth-games/62428237
1859,sports,Commonwealth Games: Table Tennis - Mixed Doubl...,Results from preliminary rounds can be found o...,/sport/commonwealth-games/62428241
2219,sports,Commonwealth Games: Badminton - Mixed Doubles ...,Results from preliminary rounds can be found o...,/sport/commonwealth-games/62409920
2352,sports,Commonwealth Games: Badminton - Women's Single...,Results from preliminary rounds can be found o...,/sport/commonwealth-games/62406473
2784,sports,Commonwealth Games: Table Tennis - Men's Singl...,Results from preliminary rounds can be found o...,/sport/commonwealth-games/62407300
2943,sports,Commonwealth Games: Badminton - Men's Doubles ...,Results from preliminary rounds can be found o...,/sport/commonwealth-games/62428244


In [220]:
train_nodup=train.drop_duplicates(subset=['headline'], keep='first')


In [221]:
train_nodup

,category,headline,text,url
0,sports,"L'Ouganda à l'assaut des ""fimbu"" de la RDC","L'Ouganda, placé 79e au classement FIFA le 4 a...",https://www.bbc.com/afrique/sports-48730747
1,business,Stopper la détérioration de l’environnement po...,La responsable de la biodiversité des Nations ...,https://www.bbc.com/afrique/articles/cevdz4xrp42o
2,sports,Coupe d'Afrique des nations 2022 : le sélectio...,"Le sélectionneur de la Sierra Leone, John Keis...",https://www.bbc.com/afrique/region-59881445
3,business,Tissus africains : pourquoi les teintureries h...,"Depuis plus de six siècles, une vaste zone sit...",https://www.bbc.com/afrique/region-61202726
4,business,Les revenus pendant la pandémie des dix hommes...,"Pendant la pandémie de coronavirus, la richess...",https://www.bbc.com/afrique/monde-55812018
...,...,...,...,...
1471,sports,C1: Le PSG va disputer sa toute première finale,Le Paris SG s'est qualifié pour sa toute premi...,https://www.bbc.com/afrique/sports-53829969
1472,sports,"Aubameyang invité à rejoindre un ""club plus am...","L'attaquant gabonais d'Arsenal, dont le contra...",https://www.bbc.com/afrique/sports-52262194
1473,politics,Guerre Ukraine - Russie : qui est Sergey Surov...,La désignation de Sergey Surovikin pour dirige...,https://www.bbc.com/afrique/monde-63228038
1474,technology,"Svetlana Jitomirskaya, la mathématicienne à l'...",Il existe un type de papillon qui captive le p...,https://www.bbc.com/afrique/monde-63956646


In [222]:
dev[dev.iloc[:,1].duplicated(keep=False).values]

,category,headline,text,url


In [223]:
dev_nodup=dev.drop_duplicates(subset=['headline'], keep='first')
 

In [224]:
dev_nodup

,category,headline,text,url
0,business,Environnement : comment l'énergie verte illimi...,Que ferions-nous d'un approvisionnement abonda...,https://www.bbc.com/afrique/monde-63182609
1,sports,Le Quarterback Patrick Mahomes signe le plus g...,La star de la NFL Patrick Mahomes prolonge d...,https://www.bbc.com/afrique/sports-53327143
2,politics,Guerre Ukraine - Russie : pourquoi la Turquie ...,La Suède et la Finlande ont officiellement pré...,https://www.bbc.com/afrique/monde-61507629
3,health,"Covid : la mucormycose, le très dangereux ""cha...","Samedi matin, le Dr Akshay Nair, un chirurgien...",https://www.bbc.com/afrique/monde-54154451
4,health,Sida: huit mythes sur le VIH,L'infection à virus de l'immunodéficience huma...,https://www.bbc.com/afrique/monde-46375213
...,...,...,...,...
206,politics,L'après Condé : qui participera aux consultati...,"La junte militaire, désormais au pouvoir à Con...",https://www.bbc.com/afrique/monde-58544800
207,business,Guerre Ukraine - Russie : l'économie russe a j...,"Le magazine britannique The Economist, dans so...",https://www.bbc.com/afrique/monde-60940836
208,health,Coronavirus : Pourquoi il est si difficile de ...,Les anniversaires importants sont célébrés par...,https://www.bbc.com/afrique/monde-52424729
209,sports,Marcus Rashford: la victoire d’un footballeur ...,L'attaquant de Manchester United Marcus Rashfo...,https://www.bbc.com/afrique/sports-53092788


In [227]:
#We take X examples from each label, and the rest become the unlabeled data-set.
classes = train['category'].unique()
samples_per_class=20 #5 10 20 50
# create an empty list to store the randomly selected samples
random_samples = []

# iterate through each class
for class_name in classes:
    # select all rows with the current class
    class_rows = train_nodup[train_nodup['category'] == class_name]
    # randomly select X rows from the class_rows DataFrame
    random_class_samples = class_rows.sample(samples_per_class)
    # add the randomly selected samples to the list
    random_samples.append(random_class_samples)
random_samples_df = pd.concat(random_samples).sample(frac=1)
#new_train=random_samples_df.reset_index(drop=True)
unlabeled_train = train.drop(new_train.index)
random_samples_df
#unlabeled_train

,category,headline,text,url
853,sports,Didier Drogba: comment l'attaquant a aidé à st...,"Le stade Al-Merrikh, dans la deuxième plus gra...",https://www.bbc.com/afrique/sports-52120608
1046,politics,Hissène Habré retourne en prison après l'expir...,"L'ancien président tchadien, Hissein Habré, co...",https://www.bbc.com/afrique/region-52960259
926,politics,Violentes manifestations en Sierra Leone : des...,Des civils et quatre policiers ont été tués lo...,https://www.bbc.com/afrique/region-62504476
750,business,La RD Congo rejoint la Communauté économique d...,La République démocratique du Congo a rejoint ...,https://www.bbc.com/afrique/region-60902893
363,health,Covid : quels sont les symptômes du variant De...,"Le variant Delta du coronavirus, considéré com...",https://www.bbc.com/afrique/monde-57498915
...,...,...,...,...
1094,business,"La cola, une noix aux 'vertus économiques' ?",La noix de cola est traditionnellement échangé...,https://www.bbc.com/afrique/monde-49257031
1164,politics,Cinq choses à savoir sur l'épouse de l'émir de...,"La princesse Haya Bint al-Hussein, épouse de l...",https://www.bbc.com/afrique/monde-48859552
10,politics,Qui est Saleh Kebzabo le nouveau Premier minis...,M﻿amadou Faye B﻿BC Afrique La tâche ne sera ap...,https://www.bbc.com/afrique/articles/c0w5nyzxl3wo
808,health,Covid : le passeport vaccinal de l'UE et ses c...,Le passeport vaccinal de l'UE - le certificat ...,https://www.bbc.com/afrique/monde-57683660


In [180]:
#new_train.columns = [0, 1, 2,3]
#unlabeled_train.columns = [0, 1, 2,3]

In [181]:
random_samples = []
samples_per_class=5
for class_name in classes:
    class_rows = dev_nodup[dev_nodup['category'] == class_name]
    random_class_samples = class_rows.sample(samples_per_class)
    random_samples.append(random_class_samples)
random_samples_df = pd.concat(random_samples).sample(frac=1)
new_dev=random_samples_df.reset_index(drop=True)
#new_dev.columns = [0, 1, 2,3]

In [182]:
unlabeled_train.insert(0, 'unlabeled', 'unlabeled')
unlabeled_train.reset_index(drop=True)

,unlabeled,category,headline,text,url
0,unlabeled,business,"Calinou’nou, la société qui veut redonner aux ...","« J'étais malade, j'ai pris une semaine, à mon...",https://www.bbc.com/afrique/region-50197742
1,unlabeled,health,Ce que les Japonais peuvent nous apprendre sur...,Le Japon a l'une des populations les plus âgée...,https://www.bbc.com/afrique/region-55619408
2,unlabeled,technology,Y-a-t'il un espion caché dans votre poche?,"Pour de nombreuses personnes, leur smartphone ...",https://www.bbc.com/afrique/monde-48828261
3,unlabeled,health,Ebola: ce que l'on sait de la résurgence de la...,La République démocratique du Congo annonce di...,https://www.bbc.com/afrique/region-55980559
4,unlabeled,politics,Le Danemark suspend son aide à la Tanzanie apr...,"Le Danemark suspend 9,8 millions de dollars d'...",https://www.bbc.com/afrique/region-46221076
...,...,...,...,...,...
1371,unlabeled,sports,C1: Le PSG va disputer sa toute première finale,Le Paris SG s'est qualifié pour sa toute premi...,https://www.bbc.com/afrique/sports-53829969
1372,unlabeled,sports,"Aubameyang invité à rejoindre un ""club plus am...","L'attaquant gabonais d'Arsenal, dont le contra...",https://www.bbc.com/afrique/sports-52262194
1373,unlabeled,politics,Guerre Ukraine - Russie : qui est Sergey Surov...,La désignation de Sergey Surovikin pour dirige...,https://www.bbc.com/afrique/monde-63228038
1374,unlabeled,technology,"Svetlana Jitomirskaya, la mathématicienne à l'...",Il existe un type de papillon qui captive le p...,https://www.bbc.com/afrique/monde-63956646


In [183]:
unlabeled_train.drop(columns='category', inplace=True)
unlabeled_train

,unlabeled,headline,text,url
100,unlabeled,"Calinou’nou, la société qui veut redonner aux ...","« J'étais malade, j'ai pris une semaine, à mon...",https://www.bbc.com/afrique/region-50197742
101,unlabeled,Ce que les Japonais peuvent nous apprendre sur...,Le Japon a l'une des populations les plus âgée...,https://www.bbc.com/afrique/region-55619408
102,unlabeled,Y-a-t'il un espion caché dans votre poche?,"Pour de nombreuses personnes, leur smartphone ...",https://www.bbc.com/afrique/monde-48828261
103,unlabeled,Ebola: ce que l'on sait de la résurgence de la...,La République démocratique du Congo annonce di...,https://www.bbc.com/afrique/region-55980559
104,unlabeled,Le Danemark suspend son aide à la Tanzanie apr...,"Le Danemark suspend 9,8 millions de dollars d'...",https://www.bbc.com/afrique/region-46221076
...,...,...,...,...
1471,unlabeled,C1: Le PSG va disputer sa toute première finale,Le Paris SG s'est qualifié pour sa toute premi...,https://www.bbc.com/afrique/sports-53829969
1472,unlabeled,"Aubameyang invité à rejoindre un ""club plus am...","L'attaquant gabonais d'Arsenal, dont le contra...",https://www.bbc.com/afrique/sports-52262194
1473,unlabeled,Guerre Ukraine - Russie : qui est Sergey Surov...,La désignation de Sergey Surovikin pour dirige...,https://www.bbc.com/afrique/monde-63228038
1474,unlabeled,"Svetlana Jitomirskaya, la mathématicienne à l'...",Il existe un type de papillon qui captive le p...,https://www.bbc.com/afrique/monde-63956646


In [135]:

#We take X examples from each label, and the rest become the unlabeled data-set.
#Instead, we could also make the new training data-set following the proportion of examples per label.
#PVP pattern-verbalizer pair (PVP)

In [184]:
new_train.to_csv("/home/sanala/Juputer try/PET/data-fsl/fra/20sample/train.tsv", sep="\t", index=False  ,header=False)
unlabeled_train.to_csv("/home/sanala/Juputer try/PET/data-fsl/fra/20sample/unlabeled.tsv", sep="\t", index=False,header=False)
new_dev.to_csv("/home/sanala/Juputer try/PET/data-fsl/fra/20sample/dev.tsv", sep="\t", index=False,header=False)

### Train your own PET



In [224]:
%cd /home/sanala/Juputer try/pet/


/home/sanala/Juputer try/pet
